### 설문 조사 데이터를 오버샘플링 하기 위한 코드
- 샘플링 수를 늘려보자 !

#### 필요한 라이브러리 로드

In [39]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
# 랜덤 오버 샘플링을 위한 라이브러리
from imblearn.over_sampling import RandomOverSampler

#### 파일 위치 지정

In [40]:
file_path = './refer/output/'

#### 설문 데이터 전처리

In [41]:
def survey_processing(file_path = f'{file_path}survey.csv', output_path = f'{file_path}survey_preprocessed.csv'):
    # CSV 파일을 불러옵니다.
    survey_df = pd.read_csv(file_path)
    
    # 우선순위에 대한 가중치 매핑을 정의합니다.
    priority_weights = {
        "건강": "health",
        "여가시간": "free_time",
        "학업 및 자기계발": "edu",
        "업무": "work",
        "집안일": "chores",
        None: "category_else"
    }
    
    # 각 우선순위 컬럼에 대해 가중치를 적용합니다.
    for i in range(1, 6):
        column_name = f"본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [{i} 순위]"
        survey_df[column_name] = survey_df[column_name].map(priority_weights)
    
    # 필요한 컬럼들만 선택하고 리네임합니다.
    columns_to_keep = [
        "설문자의 나이(만)는 어떻게 되십니까?",
        "설문자의 해당사항을 체크해주세요.",
        "설문자의 MBTI는 무엇입니까?",
        "설문자의 성별은 어떻게 되십니까?",
        "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [1 순위]",
        "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [2 순위]",
        "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [3 순위]",
        "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [4 순위]",
        "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [5 순위]"
    ]
    
    survey_df = survey_df[columns_to_keep]
    
    # 컬럼명을 변환합니다.
    column_mapping = {
        "설문자의 나이(만)는 어떻게 되십니까?": "age",
        "설문자의 해당사항을 체크해주세요.": "job",
        "설문자의 MBTI는 무엇입니까?": "mbti",
        "설문자의 성별은 어떻게 되십니까?": "gender",
        "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [1 순위]": "priority_1",
        "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [2 순위]": "priority_2",
        "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [3 순위]": "priority_3",
        "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [4 순위]": "priority_4",
        "본인이 생각하는 일과 별 중요도를 우선순위를 정하여 체크해주세요. (각 순위 별로 하나씩만 체크해주세요) [5 순위]": "priority_5"
    }
    
    survey_df.rename(columns=column_mapping, inplace=True)
    
    # 새로운 컬럼 'category_else'를 추가하고 모든 값을 0으로 설정합니다.
    survey_df["category_else"] = 0
    
    # 나이, 직업, MBTI 컬럼을 숫자 코드로 변환합니다.
    age_mapping = {
        "0~14세": (0, 14),
        "15~19세": (15, 19),
        "20~24세": (20, 24),
        "25~30세": (25, 30),
        "31세 이상": (31, 45)
    }
    job_mapping = {
        "초/중학생": "000",
        "고등학생": "001",
        "대학생 / 저학년 (1-2학년)": "002",
        "대학생 / 고학년(3-4학년)": "003",
        "구직자": "004",
        "직장인": "005",
        "자영업자": "006",
        "프리랜서": "007",
        "주부": "008",
        "기타": "009"
    }
    mbti_mapping = {
        "INTJ": "00",
        "INTP": "01",
        "ENTJ": "02",
        "ENTP": "03",
        "INFJ": "04",
        "INFP": "05",
        "ENFJ": "06",
        "ENFP": "07",
        "ISTJ": "08",
        "ISFJ": "09",
        "ESTJ": "10",
        "ESFJ": "11",
        "ISTP": "12",
        "ISFP": "13",
        "ESTP": "14",
        "ESFP": "15"
    }
    gender_mapping = {
        "남": 0,
        "여": 1
    }

    # 나이, 직업, MBTI, 성별 컬럼을 숫자 코드로 변환합니다.
    survey_df['age'] = survey_df['age'].map(lambda x: np.random.randint(age_mapping[x][0], age_mapping[x][1] + 1))
    survey_df['job'] = survey_df['job'].map(job_mapping)
    survey_df['mbti'] = survey_df['mbti'].str.upper().map(mbti_mapping)
    survey_df['gender'] = survey_df['gender'].map(gender_mapping)

    # 우선순위별로 각 항목에 가중치를 적용한 값을 할당합니다.
    work_col = []
    edu_col = []
    free_time_col = []
    health_col = []
    chores_col = []

    for _, row in survey_df.iterrows():
        work = 0
        edu = 0
        free_time = 0
        health = 0
        chores = 0

        for i in range(1, 6):
            if row[f'priority_{i}'] == 'work':
                work = 1 - (i - 1) * 0.25
            elif row[f'priority_{i}'] == 'edu':
                edu = 1 - (i - 1) * 0.25
            elif row[f'priority_{i}'] == 'free_time':
                free_time = 1 - (i - 1) * 0.25
            elif row[f'priority_{i}'] == 'health':
                health = 1 - (i - 1) * 0.25
            elif row[f'priority_{i}'] == 'chores':
                chores = 1 - (i - 1) * 0.25
        
        work_col.append(work)
        edu_col.append(edu)
        free_time_col.append(free_time)
        health_col.append(health)
        chores_col.append(chores)
    
    survey_df['work'] = work_col
    survey_df['edu'] = edu_col
    survey_df['free_time'] = free_time_col
    survey_df['health'] = health_col
    survey_df['chores'] = chores_col
    
    # 사용하지 않는 우선순위 컬럼 삭제
    survey_df.drop(columns=['priority_1', 'priority_2', 'priority_3', 'priority_4', 'priority_5'], inplace=True)
    
    # 컬럼 순서 재정렬
    survey_df = survey_df[['age', 'job', 'mbti', 'gender', 'work', 'edu', 'free_time', 'health', 'chores', 'category_else']]
    
    # 수정된 데이터프레임을 새로운 CSV 파일로 저장합니다.
    survey_df.to_csv(output_path, index=False)

In [42]:
survey_processing()  # 함수를 호출하여 변환을 수행합니다.

#### 원본 데이터 단순 복제로 데이터 수를 10만개로 늘리기

In [43]:
# 데이터 로드
data = pd.read_csv(f'{file_path}survey_preprocessed.csv')

# 원하는 샘플 수
target_samples = 100000
replication_factor = target_samples // len(data)  # 필요한 복제 횟수 계산
additional_samples = target_samples % len(data)   # 추가로 필요한 샘플 수

# 데이터 복제
oversampled_data = pd.concat([data] * replication_factor + [data.iloc[:additional_samples]])

# 결과 저장
oversampled_data.to_csv(f'{file_path}survey_replicated.csv', index=False)

print(f"데이터 복제 완료: 총 샘플 수 {len(oversampled_data)}")


데이터 복제 완료: 총 샘플 수 100000


#### 랜덤 오버 샘플링을 통해 전체 데이터의 수를 늘리기

- 단, 랜덤 오버 샘플링은 이산형 데이터에 사용이 가능
- 따라서 '0', '0.25', '0.5', '0.75', '1'을 범주화하여 제한을 우회
- 이후 다시 범주형 데이터를 수치로 롤백하여 원본 데이터의 형식을 유지
- 이 외의 숫자가 혹시 포함되어 있다면 디버깅을 위해 -1로 설정

- 랜덤 오버 샘플링을 위해 각 열을 독립적으로 오버 샘플링 후 다시 합쳐주는 방식 선택

- 하지만 지금까지는 실패~
    - 각 열을 독립적으로 오버 샘플링 하다보니 각 열의 인덱스를 합쳐야 NA가 없음
    - 그러나 랜덤으로 샘플링 하기 때문에 각 파일별 인덱스 수가 달라짐
    - 따라서 망함 :,<

In [44]:
# 메모리 사용을 줄이기 위해 범주에 대해 개별 오버 샘플링 후 저장
'''
def discretize(value):
    if value == 0:
        return 'Very Low'
    elif value == 0.25:
        return 'Low'
    elif value == 0.5:
        return 'Medium'
    elif value == 0.75:
        return 'High'
    elif value == 1:
        return 'Very High'
    return 'Unknown'

def continuousize(category):
    mapping = {'Very Low': 0, 'Low': 0.25, 'Medium': 0.5, 'High': 0.75, 'Very High': 1}
    return mapping.get(category, -1)

# 데이터 로드
original_data = pd.read_csv(f'{file_path}survey_replicated.csv')
categories = ['work', 'edu', 'free_time', 'health', 'chores']

# 범주화 및 원-핫 인코딩
X_encoded = pd.get_dummies(original_data.drop(categories, axis=1))
ros = RandomOverSampler(random_state=42)

# 각 범주에 대해 오버샘플링 적용 및 저장
for category in categories:
    y = original_data[category].apply(discretize)
    X_resampled, y_resampled = ros.fit_resample(X_encoded, y)
    y_resampled = y_resampled.apply(continuousize)
    df_resampled = pd.DataFrame(X_resampled, columns=X_encoded.columns)
    df_resampled[category] = y_resampled
    df_resampled.to_csv(f'{file_path}survey_random_oversampled_{category}.csv', index=False)
    print(f"오버샘플링 데이터 저장 완료: survey_random_oversampled_{category}.csv, 샘플 수: {len(df_resampled)}")

print("모든 범주에 대한 오버샘플링 데이터 저장 완료")
'''

'\ndef discretize(value):\n    if value == 0:\n        return \'Very Low\'\n    elif value == 0.25:\n        return \'Low\'\n    elif value == 0.5:\n        return \'Medium\'\n    elif value == 0.75:\n        return \'High\'\n    elif value == 1:\n        return \'Very High\'\n    return \'Unknown\'\n\ndef continuousize(category):\n    mapping = {\'Very Low\': 0, \'Low\': 0.25, \'Medium\': 0.5, \'High\': 0.75, \'Very High\': 1}\n    return mapping.get(category, -1)\n\n# 데이터 로드\noriginal_data = pd.read_csv(f\'{file_path}survey_replicated.csv\')\ncategories = [\'work\', \'edu\', \'free_time\', \'health\', \'chores\']\n\n# 범주화 및 원-핫 인코딩\nX_encoded = pd.get_dummies(original_data.drop(categories, axis=1))\nros = RandomOverSampler(random_state=42)\n\n# 각 범주에 대해 오버샘플링 적용 및 저장\nfor category in categories:\n    y = original_data[category].apply(discretize)\n    X_resampled, y_resampled = ros.fit_resample(X_encoded, y)\n    y_resampled = y_resampled.apply(continuousize)\n    df_resampled = pd.Da

In [45]:
# 모든 열에 처리시 메모리 사용량이 지나치게 높기에 위 코드를 통해 각 범주별 개별 오버샘플링 적용 후 합치는 방식 선택
'''
# 데이터 로드 및 함수 정의
def discretize(value):
    if value == 0:
        return 'Very Low'
    elif value == 0.25:
        return 'Low'
    elif value == 0.5:
        return 'Medium'
    elif value == 0.75:
        return 'High'
    elif value == 1:
        return 'Very High'
    return 'Unknown'

def continuousize(category):
    mapping = {'Very Low': 0, 'Low': 0.25, 'Medium': 0.5, 'High': 0.75, 'Very High': 1}
    return mapping.get(category, -1)

original_data = pd.read_csv(f'{file_path}survey_replicated.csv')
categories = ['work', 'edu', 'free_time', 'health', 'chores']

# 범주화 및 원-핫 인코딩
X_encoded = pd.get_dummies(original_data.drop(categories, axis=1))
oversampled_dataframes = []
ros = RandomOverSampler(random_state=42)

# 각 범주에 대해 오버샘플링 적용
for category in categories:
    y = original_data[category].apply(discretize)
    X_resampled, y_resampled = ros.fit_resample(X_encoded, y)
    y_resampled = y_resampled.apply(continuousize)
    df_resampled = pd.DataFrame(X_resampled, columns=X_encoded.columns)
    df_resampled[category] = y_resampled
    oversampled_dataframes.append(df_resampled)

# 모든 결과를 하나의 DataFrame으로 병합
final_df = oversampled_dataframes[0]
for df in oversampled_dataframes[1:]:
    final_df = final_df.merge(df, on=list(X_encoded.columns), how='inner')

# 반복적 오버샘플링으로 추가 증가
for _ in range(200):  # 반복 횟수에 따라 조정 가능
    final_df, _ = ros.fit_resample(final_df, final_df.columns)

# 결과 저장
final_df.to_csv(f'{file_path}survey_random_oversampled.csv', index=False)
print(f"오버샘플링 데이터 저장 완료: survey_random_oversampled.csv, 샘플 수: {len(final_df)}")
'''

'\n# 데이터 로드 및 함수 정의\ndef discretize(value):\n    if value == 0:\n        return \'Very Low\'\n    elif value == 0.25:\n        return \'Low\'\n    elif value == 0.5:\n        return \'Medium\'\n    elif value == 0.75:\n        return \'High\'\n    elif value == 1:\n        return \'Very High\'\n    return \'Unknown\'\n\ndef continuousize(category):\n    mapping = {\'Very Low\': 0, \'Low\': 0.25, \'Medium\': 0.5, \'High\': 0.75, \'Very High\': 1}\n    return mapping.get(category, -1)\n\noriginal_data = pd.read_csv(f\'{file_path}survey_replicated.csv\')\ncategories = [\'work\', \'edu\', \'free_time\', \'health\', \'chores\']\n\n# 범주화 및 원-핫 인코딩\nX_encoded = pd.get_dummies(original_data.drop(categories, axis=1))\noversampled_dataframes = []\nros = RandomOverSampler(random_state=42)\n\n# 각 범주에 대해 오버샘플링 적용\nfor category in categories:\n    y = original_data[category].apply(discretize)\n    X_resampled, y_resampled = ros.fit_resample(X_encoded, y)\n    y_resampled = y_resampled.apply(continuo

In [46]:
'''
import glob
import os

# 모든 CSV 파일의 경로를 리스트로 로드
csv_files = glob.glob(os.path.join(file_path, 'survey_random*.csv'))

# CSV 파일 경로 출력 (디버깅용)
print("CSV files found:", csv_files)

# CSV 파일이 존재하는지 확인
if not csv_files:
    raise ValueError("No CSV files found in the specified directory.")

# 모든 CSV 파일을 읽어 하나의 데이터프레임으로 병합
dataframes = []
for file in csv_files:
    try:
        df = pd.read_csv(file)
        dataframes.append(df)
    except Exception as e:
        print(f"Error reading {file}: {e}")

# 데이터프레임 병합
if dataframes:
    merged_dataframe = pd.concat(dataframes, ignore_index=True)
    # 합쳐진 데이터프레임을 새로운 CSV 파일로 저장
    merged_dataframe.to_csv(f"{file_path}merged_survey_data.csv", index=False)
else:
    raise ValueError("No valid CSV files to concatenate.")
'''

'\nimport glob\nimport os\n\n# 모든 CSV 파일의 경로를 리스트로 로드\ncsv_files = glob.glob(os.path.join(file_path, \'survey_random*.csv\'))\n\n# CSV 파일 경로 출력 (디버깅용)\nprint("CSV files found:", csv_files)\n\n# CSV 파일이 존재하는지 확인\nif not csv_files:\n    raise ValueError("No CSV files found in the specified directory.")\n\n# 모든 CSV 파일을 읽어 하나의 데이터프레임으로 병합\ndataframes = []\nfor file in csv_files:\n    try:\n        df = pd.read_csv(file)\n        dataframes.append(df)\n    except Exception as e:\n        print(f"Error reading {file}: {e}")\n\n# 데이터프레임 병합\nif dataframes:\n    merged_dataframe = pd.concat(dataframes, ignore_index=True)\n    # 합쳐진 데이터프레임을 새로운 CSV 파일로 저장\n    merged_dataframe.to_csv(f"{file_path}merged_survey_data.csv", index=False)\nelse:\n    raise ValueError("No valid CSV files to concatenate.")\n'

#### 랜덤 오버 샘플링을 통해 늘어난 데이터 수를 바탕으로 smote 적용

In [47]:
# 데이터 로드
original_data = pd.read_csv(f'{file_path}survey_replicated.csv')

# 데이터 스케일링 및 클러스터링
scaler = StandardScaler()
y_scaled = scaler.fit_transform(original_data[['work', 'edu', 'free_time', 'health', 'chores']])

kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(y_scaled)


# SMOTE 적용 (연속형 데이터 포함)
# n_neighbors 값 조정: 가장 작은 클러스터의 크기보다 작게 설정
min_cluster_size = min(pd.Series(clusters).value_counts())
n_neighbors = max(min(2, min_cluster_size - 1), 1)  # 최소 1, 최대 (가장 작은 클러스터 크기 - 1) 사이
smote = SMOTE(random_state=42, k_neighbors=n_neighbors)
X_resampled, clusters_resampled = smote.fit_resample(original_data, clusters)

# 연속형 데이터 복원: 각 클러스터의 중심을 사용하여 연속형 데이터를 복원
cluster_centers = scaler.inverse_transform(kmeans.cluster_centers_)
oversampled_y = [cluster_centers[cluster] for cluster in clusters_resampled]

# 오버샘플링된 데이터 프레임 생성
oversampled_df = pd.DataFrame(X_resampled, columns=original_data.columns)
oversampled_df[['work', 'edu', 'free_time', 'health', 'chores']] = pd.DataFrame(oversampled_y, columns=['work', 'edu', 'free_time', 'health', 'chores'])

# 결과 저장 및 출력
oversampled_df.to_csv(f'{file_path}survey_oversampled.csv', index=False)
print("오버샘플링 데이터 저장 완료: survey_oversampled.csv")


오버샘플링 데이터 저장 완료: survey_oversampled.csv


#### 오버 샘플링 된 데이터들을 정제하는 과정

In [48]:
'''
# 오버샘플링된 데이터 로드
oversampled_data = pd.read_csv(f'{file_path}survey_oversampled.csv')

# 'job' 관련 열 추출 및 변환
job_columns = [col for col in oversampled_data.columns if col.startswith('job_')]
oversampled_data['job'] = oversampled_data[job_columns].idxmax(axis=1).apply(lambda x: x.split('_')[1] if pd.notna(x) else None)

# 'mbti' 관련 열 추출 및 변환
mbti_columns = [col for col in oversampled_data.columns if col.startswith('mbti_')]
oversampled_data['mbti'] = oversampled_data[mbti_columns].idxmax(axis=1).apply(lambda x: x.split('_')[1] if pd.notna(x) else None)

# 불필요한 원-핫 인코딩 열 제거
oversampled_data.drop(columns=job_columns + mbti_columns, inplace=True)

# 결과 저장
oversampled_data.to_csv(f'{file_path}survey_oversampled_fixed.csv', index=False)

print("데이터 업데이트 완료: survey_oversampled_fixed.csv")
'''

'\n# 오버샘플링된 데이터 로드\noversampled_data = pd.read_csv(f\'{file_path}survey_oversampled.csv\')\n\n# \'job\' 관련 열 추출 및 변환\njob_columns = [col for col in oversampled_data.columns if col.startswith(\'job_\')]\noversampled_data[\'job\'] = oversampled_data[job_columns].idxmax(axis=1).apply(lambda x: x.split(\'_\')[1] if pd.notna(x) else None)\n\n# \'mbti\' 관련 열 추출 및 변환\nmbti_columns = [col for col in oversampled_data.columns if col.startswith(\'mbti_\')]\noversampled_data[\'mbti\'] = oversampled_data[mbti_columns].idxmax(axis=1).apply(lambda x: x.split(\'_\')[1] if pd.notna(x) else None)\n\n# 불필요한 원-핫 인코딩 열 제거\noversampled_data.drop(columns=job_columns + mbti_columns, inplace=True)\n\n# 결과 저장\noversampled_data.to_csv(f\'{file_path}survey_oversampled_fixed.csv\', index=False)\n\nprint("데이터 업데이트 완료: survey_oversampled_fixed.csv")\n'

In [49]:
# 마지막으로 value 수정
fix_data = pd.read_csv(f'{file_path}survey_oversampled.csv')

# 열 순서 재배열
columns_ordered = ['age', 'gender', 'job', 'mbti', 'work', 'edu', 'free_time', 'health', 'chores']
fix_data = fix_data[columns_ordered]

fix_data.to_csv(f'{file_path}survey_data.csv')

In [50]:
fix_data.describe()

,age,gender,job,mbti,work,edu,free_time,health,chores
count,109095.000000,109095.000000,109095.000000,109095.000000,109095.000000,109095.000000,109095.000000,109095.000000,109095.000000
mean,29.457253,0.608085,4.375618,7.432036,0.638429,0.426707,0.477849,0.659269,0.205103
std,7.433338,0.488180,1.580834,4.507275,0.191892,0.211148,0.249885,0.246948,0.202161
min,14.000000,0.000000,0.000000,0.000000,0.368404,0.265633,0.124991,0.312485,0.012505
25%,25.000000,0.000000,3.000000,4.000000,0.368404,0.265633,0.124991,0.312485,0.012505
50%,28.000000,1.000000,4.000000,7.000000,0.750027,0.289492,0.637495,0.796889,0.118418
75%,34.000000,1.000000,5.000000,12.000000,0.796855,0.724997,0.671059,0.868432,0.484385
max,45.000000,1.000000,8.000000,15.000000,0.796855,0.724997,0.671059,0.868432,0.484385


----------------------------------------------------

### 데이터를 DB에 저장하기 위해 정제

In [51]:
# 파일 경로
path = './survey_oversampled_fixed.csv'

# CSV 파일 읽기
data = pd.read_csv(path)

# 나이 범위를 age_range 컬럼으로 추가
bins = [0, 20, 25, 30, 100]
labels = ['0-19', '20-24', '25-29', '30-']
data['age_range'] = pd.cut(data['age'], bins=bins, labels=labels, right=False)

# age_range 행을 기준으로 분류하여 각 변수의 가중치 평균 계산
grouped_means = data.groupby('age_range')[['work', 'edu', 'free_time', 'health', 'chores']].mean()

# 결과 출력
grouped_means

FileNotFoundError: [Errno 2] No such file or directory: './survey_oversampled_fixed.csv'